In [3]:
import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import pandas as pd

# Diretório que você deseja monitorar
diretorio_monitorado = 'X:\\ARQUIVOS\\3810-GTSS\\Relatorios Viasat\\2023-Set\\'

# Iniciais dos nomes de arquivo que você deseja monitorar
iniciais_arquivos_monitorados = ['TelebrasHourlyUsage-v2-']  # Adicione suas iniciais desejadas

# Diretório para registrar os resultados
diretorio_resultados = 'C:\\Users\\gustavo.costa\\Desktop\\blablaasd'

# Função para verificar o arquivo CSV
def verificar_arquivo_csv(nome_arquivo):
    try:
        # Carrega o arquivo CSV em um DataFrame usando o pandas
        df = pd.read_csv(os.path.join(diretorio_monitorado, nome_arquivo))
        
        # Verifica se há problemas (por exemplo, colunas vazias)
        # Adicione suas verificações personalizadas aqui
        
        return None  # Nenhum problema encontrado
    
    except Exception as e:
        return f'Problema no arquivo {nome_arquivo}: {str(e)}\n'

# Função para lidar com eventos de arquivo
class MeuHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return  # Ignorar diretórios
        
        nome_arquivo = os.path.basename(event.src_path)
        for inicial in iniciais_arquivos_monitorados:
            if nome_arquivo.startswith(inicial):
                print(f'Novo arquivo {event.src_path} detectado. Verificando...')
                resultado = verificar_arquivo_csv(nome_arquivo)
                if resultado:
                    with open(os.path.join(diretorio_resultados, 'arquivos_com_problemas.txt'), 'a') as arquivo_saida:
                        arquivo_saida.write(resultado)
                    print(f'Problema encontrado no arquivo {event.src_path} e registrado em "arquivos_com_problemas.txt".')
                else:
                    print(f'Nenhum problema encontrado no arquivo {event.src_path}.')

# Inicializa o observador e o manipulador
observer = Observer()
handler = MeuHandler()
observer.schedule(handler, path=diretorio_monitorado, recursive=False)

# Inicia a observação em segundo plano
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()

# Aguarda a finalização do observador
observer.join()


Novo arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230920.csv detectado. Verificando...
Problema encontrado no arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230920.csv e registrado em "arquivos_com_problemas.txt".
Novo arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230923.csv detectado. Verificando...
Problema encontrado no arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230923.csv e registrado em "arquivos_com_problemas.txt".
Novo arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230924.csv detectado. Verificando...
Problema encontrado no arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230924.csv e registrado em "arquivos_com_problemas.txt".
Novo arquivo X:\ARQUIVOS\3810-GTSS\Relatorios Viasat\2023-Set\TelebrasHourlyUsage-v2-20230922.csv detectado. Verificando...
Problema encontrad

In [16]:
import pandas as pd

# Carregue o arquivo CSV em um DataFrame
df = pd.read_csv('X:\\ARQUIVOS\\3810-GTSS\\Relatorios Viasat\\2023-ago\\SGDCPerformanceDownload20230801.csv')
# Função para converter strings no formato 'xx.xxx.xxx,xx' em float
def convert_rate_score(value):
    value = value.replace('.', '').replace(',', '.')
    return float(value)

# Aplicar a função à coluna rate_score
df['rate_score'] = df['rate_score'].apply(convert_rate_score)

# Filtrar as linhas com rate_score maior que 12,999999
result = df[df['rate_score'] > 12.999999]

# Exibir os valores de rate_score correspondentes
rate_scores = result['rate_score'].tolist()
print(rate_scores)

AttributeError: 'float' object has no attribute 'replace'